# NHANES 2021–2022: Clean Physical Activity (PAQ) data

This notebook:
- Loads NHANES **PAQ_L** (physical activity) and **DEMO_L** (demographics)
- Standardizes activity frequency units to **weekly minutes**
- Builds a small, analysis-ready table (adults only) and saves it as CSV


## 1) Setup

In [15]:
import pandas as pd
import numpy as np

# --- File paths (edit if your folder structure changes) ---
PAQ_FILE = "../data/PAQ_L.xpt"
DEMO_FILE = "../data/DEMO_L.xpt"

# NHANES sometimes uses a special floating value for missing in XPT files.
NHANES_MISSING = 5.397605e-79


## 2) Load and merge (SEQN)

In [16]:
# Load datasets
paq = pd.read_sas(PAQ_FILE).rename(columns={"SEQN": "seqn"})
demo = pd.read_sas(DEMO_FILE)

# Keep only the demographics we need and standardize column names
demo = demo[["SEQN", "RIDAGEYR", "RIAGENDR"]].rename(
    columns={"SEQN": "seqn", "RIDAGEYR": "age", "RIAGENDR": "sex"}
)

# Merge demographics into PAQ using the respondent id (SEQN/seqn)
df = paq.merge(demo, on="seqn", how="left")

# Replace NHANES special-missing value with NaN (so pandas math works as expected)
df = df.replace(NHANES_MISSING, np.nan)


## 3) Helper functions

In [17]:
# Some unit columns come in as bytes (e.g., b'D'). Decode to plain strings.
def decode_unit(x):
    if isinstance(x, (bytes, bytearray)):
        return x.decode("utf-8").strip()
    return x

UNIT_TO_WEEK = {
    "D": 7,          # days/week reported -> multiply by 7 days/week
    "W": 1,          # weeks/week reported -> multiply by 1
    "M": 1 / 4.33,   # months -> approx. weeks/month
    "Y": 1 / 52,     # years -> weeks/year
}

def weekly_minutes(freq, unit, minutes_per_day):
    """Convert (frequency, unit, minutes per day) into minutes per week."""
    if pd.isna(freq) or pd.isna(unit) or pd.isna(minutes_per_day):
        return np.nan

    mult = UNIT_TO_WEEK.get(unit)
    if mult is None:
        return np.nan

    return float(freq) * float(minutes_per_day) * mult


## 4) Feature engineering

In [18]:
# Decode unit fields so they match UNIT_TO_WEEK keys
df["PAD790U"] = df["PAD790U"].apply(decode_unit)   # moderate unit
df["PAD810U"] = df["PAD810U"].apply(decode_unit)   # vigorous unit

# Weekly minutes of moderate and vigorous activity
df["moderate_min_week"] = df.apply(
    lambda r: weekly_minutes(r["PAD790Q"], r["PAD790U"], r["PAD800"]),
    axis=1,
)

df["vigorous_min_week"] = df.apply(
    lambda r: weekly_minutes(r["PAD810Q"], r["PAD810U"], r["PAD820"]),
    axis=1,
)

# MVPA equivalent (vigorous counts double)
df["mvpa_equiv_min_week"] = df["moderate_min_week"] + 2 * df["vigorous_min_week"]

# Sedentary time: convert minutes/day to hours/day
df["sedentary_hours_day"] = df["PAD680"] / 60

# Simple outlier rule: >16 hours/day sedentary is likely invalid -> set to missing
df.loc[df["sedentary_hours_day"] > 16, "sedentary_hours_day"] = np.nan


## 5) Final clean tables (all ages + adults only)

In [19]:
# Keep only the columns we need for modeling / analysis
df_clean_all = df[
    [
        "seqn",
        "age",
        "sex",
        "moderate_min_week",
        "vigorous_min_week",
        "mvpa_equiv_min_week",
        "sedentary_hours_day",
    ]
].copy()

# Adults only (18+)
df_clean_adults = df_clean_all[df_clean_all["age"] >= 18].copy()

# (Optional) If we decide to use sex as labels instead of NHANES codes (1=Male, 2=Female):
# df_clean_adults["sex_label"] = df_clean_adults["sex"].map({1: "Male", 2: "Female"})


## 6) Quick sanity checks

In [20]:
print("All ages shape:", df_clean_all.shape)
print("Adults shape:", df_clean_adults.shape)
print("Min age (adults):", df_clean_adults["age"].min())

# Missingness rate per column (adults)
display(df_clean_adults.isna().mean().sort_values(ascending=False))

# Basic distribution (adults)
display(df_clean_adults.describe(include="all"))


All ages shape: (8153, 7)
Adults shape: (8153, 7)
Min age (adults): 18.0


mvpa_equiv_min_week    0.560039
vigorous_min_week      0.547896
moderate_min_week      0.216239
sedentary_hours_day    0.016926
seqn                   0.000000
age                    0.000000
sex                    0.000000
dtype: float64

,seqn,age,sex,moderate_min_week,vigorous_min_week,mvpa_equiv_min_week,sedentary_hours_day
count,8153.000000,8153.000000,8153.000000,6390.000000,3686.000000,3587.000000,8.015000e+03
mean,136378.394579,52.144364,1.551453,507.418987,317.738079,1050.612741,5.956020e+00
std,3440.455897,18.553060,0.497376,6461.971028,5939.236023,12409.240859,3.357891e+00
min,130378.000000,18.000000,1.000000,0.192308,0.192308,1.826923,8.996009e-81
25%,133395.000000,36.000000,1.000000,75.000000,30.000000,210.000000,3.000000e+00
50%,136419.000000,55.000000,2.000000,180.000000,60.000000,390.000000,5.000000e+00
75%,139346.000000,68.000000,2.000000,315.000000,180.000000,720.000000,8.000000e+00
max,142310.000000,80.000000,2.000000,419958.000000,349965.000000,706230.000000,1.600000e+01


## 7) Save outputs

In [21]:
OUT_ALL = "../data/paq_activity_clean.csv"
OUT_ADULTS = "../data/paq_activity_clean_adults.csv"

df_clean_all.to_csv(OUT_ALL, index=False)
df_clean_adults.to_csv(OUT_ADULTS, index=False)

print(f"Saved: {OUT_ALL}")
print(f"Saved: {OUT_ADULTS}")


Saved: ../data/paq_activity_clean.csv
Saved: ../data/paq_activity_clean_adults.csv
